In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
# import smote
# import rus

In [34]:
df = pd.read_csv("AF04.csv")
df = df.set_index(pd.to_datetime(df['date']))
df = df.drop(['date'], axis=1)
df.head()

,company_symbol,rating,change_direction,CDS_SPREAD,DD,DC,dp_BB,dp_stock,dp_CDS,z,...,BBstock,BB_shift,stock_shift,CDS_shift,date.1,stock_BB_shift,new_BB,new_CDS,new_stock,new_BBstock
date,,,,,,,,,,,,,,,,,,,,,
2014-04-23,LLL,10,0,34,82.273197,135.016527,0.000001,0.0,0.630798,1.618944,...,0.016290,0.000017,0.0,0.758377,4/23/2014,0.000017,0.000017,0.758377,0.0,0.000017
2014-04-24,LLL,10,0,35,81.751962,134.768934,0.000001,0.0,0.649351,1.618944,...,0.017374,0.000018,0.0,0.776014,4/24/2014,0.000018,0.000018,0.776014,0.0,0.000018
2014-04-25,LLL,10,0,35,81.156019,134.001400,0.000002,0.0,0.649351,1.618944,...,0.018821,0.000017,0.0,0.758377,4/25/2014,0.000017,0.000017,0.758377,0.0,0.000017
2014-04-28,LLL,10,0,35,80.804757,133.616587,0.000002,0.0,0.649351,1.618944,...,0.019544,0.000016,0.0,0.793651,4/28/2014,0.000016,0.000016,0.793651,0.0,0.000016
2014-04-29,LLL,10,0,35,81.214871,133.872096,0.000002,0.0,0.649351,1.618944,...,0.018459,0.000016,0.0,0.776014,4/29/2014,0.000016,0.000016,0.776014,0.0,0.000016


In [3]:
df = df.dropna()

In [9]:
# feature_names = ['CDS_SPREAD','DD','DC','x1','x2','x3','z','z_new']
# X = df[feature_names]
# y = df['target']
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size=0.3, 
#                                                     random_state=10)

In [45]:
target_names = ['No change', 'Downgrade']
feature_names = ['CDS_SPREAD','DC','DD','new_stock','new_BB','new_CDS','new_BBstock','z']
X = df[feature_names]
y = df['target'] ==1

In [46]:
X_train = X.loc['2012-1-1':'2015-12-31']
X_test = X.loc['2016-1-1':'2017-12-31']
y_train = y.loc['2012-1-1':'2015-12-31']
y_test = y.loc['2016-1-1':'2017-12-31']

In [47]:
def adaboost(X_train, X_test, y_train):
    model = AdaBoostClassifier(n_estimators=200, learning_rate=1.6, random_state=10)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test) 
    return y_pred
# AdaBoost
y_baseline = adaboost(X_train, X_test, y_train)
# SMOTE
sm = SMOTE(random_state=10)
X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)
y_smote = adaboost(X_train_sm, X_test, y_train_sm)
# RUS
X_full = X_train.copy()
X_full['target'] = y_train
X_maj = X_full[X_full.target==0]
X_min = X_full[X_full.target==1]
X_maj_rus = resample(X_maj,replace=False,n_samples=len(X_min),random_state=44)
X_rus = pd.concat([X_maj_rus, X_min])
X_train_rus = X_rus.drop(['target'], axis=1)
y_train_rus = X_rus.target
y_rus = adaboost(X_train_rus, X_test, y_train_rus)
# ADASYN
from imblearn.over_sampling import ADASYN
ad = ADASYN(random_state=10)
X_train_sm, y_train_sm = ad.fit_sample(X_train, y_train)
y_ad = adaboost(X_train_sm, X_test, y_train_sm)


In [48]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
target_names = ['No change', 'Downgrade']
print(feature_names, 'LR = 1.6')
print("AdaBoost")
print("Accuracy:",accuracy_score(y_test, y_baseline))
print(classification_report(y_test, y_baseline, 
                                target_names=target_names))

print("AdaBoost + Smote")
print("Accuracy:",accuracy_score(y_test, y_smote))
print(classification_report(y_test, y_smote, 
                                target_names=target_names))
print("AdaBoost + RUS")
print("Accuracy:",accuracy_score(y_test, y_rus))
print(classification_report(y_test, y_rus, 
                                target_names=target_names))
print("AdaBoost + ADASYN")
print("Accuracy:",accuracy_score(y_test, y_baseline))
print(classification_report(y_test, y_ad, 
                                target_names=target_names))

['CDS_SPREAD', 'DC', 'DD', 'new_stock', 'new_BB', 'new_CDS', 'new_BBstock', 'z'] LR = 1.6
AdaBoost
Accuracy: 0.994315915436777
              precision    recall  f1-score   support

   No change       1.00      1.00      1.00     29939
   Downgrade       0.00      0.00      0.00       145

   micro avg       0.99      0.99      0.99     30084
   macro avg       0.50      0.50      0.50     30084
weighted avg       0.99      0.99      0.99     30084

AdaBoost + Smote
Accuracy: 0.994415636218588
              precision    recall  f1-score   support

   No change       1.00      1.00      1.00     29939
   Downgrade       0.00      0.00      0.00       145

   micro avg       0.99      0.99      0.99     30084
   macro avg       0.50      0.50      0.50     30084
weighted avg       0.99      0.99      0.99     30084

AdaBoost + RUS
Accuracy: 0.4391370828347294
              precision    recall  f1-score   support

   No change       0.99      0.44      0.61     29939
   Downgrade       0.

In [49]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_baseline)
print('AdaBoost \n', confusion)
confusion = confusion_matrix(y_test, y_smote)
print('\nAdaBoost+smote \n', confusion)
confusion = confusion_matrix(y_test, y_rus)
print('\nAdaBoost+RUS \n', confusion)
confusion = confusion_matrix(y_test, y_ad)
print('\nAdaBoost +ADASYN \n', confusion)

AdaBoost 
 [[29913    26]
 [  145     0]]

AdaBoost+smote 
 [[29916    23]
 [  145     0]]

AdaBoost+RUS 
 [[13152 16787]
 [   86    59]]

AdaBoost +ADASYN 
 [[29656   283]
 [  145     0]]


['CDS_SPREAD', 'dp_BB', 'dp_stock', 'dp_CDS', 'DD', 'DC', 'z', 'z_new'] 